In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from nltk.corpus import wordnet
syns = wordnet.synsets("program")

# things we need for keras
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
import random
import re
import unicodedata
import sys

Using TensorFlow backend.


In [2]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [3]:
# medical conditions training data
training_data = []

import csv

# adds every medical condition along with associated info from csv file
with open('Disease Database NEW 2 CSV.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            training_data.append({"class":row[1], "synonyms":row[2], 
                                  "sentence":row[3]+';'+row[6], "treatments":row[4], "danger level":row[5]})
        line_count += 1

print ("%s total classes in training data" % len(training_data))

for term in training_data:
    print(term["class"] + " | " + term["synonyms"] + " | " + term["sentence"] + 
          " | " + term["treatments"] + " | " + term["danger level"])

14 total classes in training data
Common Cold | Upper Respiratory Tract;Nose and Throat Infection | Hi, I have a runny nose all the time and I don't know what to do about it. I was sniffing all the time in the past few days and it was getting very annoying;One of my nausal was blocked and I couldn't breath properly;My throat hurts after a cold shower last night, I am not sure what to do about it;It feels like there was something in my throat and I've been dry coughing for several days already;I recently developed a condition that requires frequent washroom visits througout the day;I am so tired this morning, every single one of my muscle hurts so much;All my joints are sour since last morning;I feel so cold all the time, I sneezed all the time;I am not so conformatble;I caught a cold;I am shiverring;My throat hurts;I an tired all the time and I don't want to move;Runny or stuffy nose;sore throat;cough;congestion;body aches;sneezing | Stay hydrated;rest;sooth a sore throat using saltwat

In [4]:
# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
     return text.translate(tbl)

# method to slice strings
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
         return my_str[index:] 
    else :
         raise Exception('Sub string not found!')

# # find synonyms of all the words in a sentence
# def find_syns(sentence):
#     words = []
#     for word in sentence.split():
#         if word != '':
#             synonyms = []
#             for syn in wordnet.synsets(word):
#                 for l in syn.lemmas():
#                     if l.name() not in synonyms and l.name() != word:
#                         synonyms.append(l.name())
#             word = (word + ' ' + ' '.join(synonyms)).strip()
#             words.append(word)
#     print("SYNONYMS:",words)
#     return words

# # clean all synonyms and words
# def clean_syns(arg):
#     clean_synonyms = []
#     for syn_set in synonyms:
#         syn_set = ' '.join(s for s in syn_set.split() if not any(c.isdigit() for c in s))
#         syn_set = re.sub(r'\d+', '', syn_set)
#         syn_set = syn_set.replace("_"," ")
#         syn_set = remove_punctuation(syn_set)
#         clean_synonyms.append(syn_set)
#     print("CLEAN SYNONYMS: ",clean_synonyms)
#     return clean_synonyms

In [5]:
import string

words = []
words_total = []
classes = []
classes_details = []
documents = []
ignore_words = ['?']

# loop through each description per symptom
for pattern in training_data:
    symptoms = pattern['sentence']
    word_list = symptoms.split(';')
    print("WORD_LIST:",word_list)
    word_list_new = []
    for word in word_list:
        word = remove_punctuation(word)
        word = word.strip().lower()
        word_list_new.append(word)
    print("WORD_LIST_NEW:",word_list_new)
    joined_symps = ' '.join(s for s in word_list_new)
    print("joined_symps:",joined_symps)
    
    # tokenize each word in the description
    w = nltk.word_tokenize(joined_symps)
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        classes_details.append({"class":pattern['class'], "synonyms":pattern['synonyms'],
                                "treatments":pattern['treatments'], "danger level":pattern['danger level']})
    
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print ("classes_details", classes_details)

WORD_LIST: ["Hi, I have a runny nose all the time and I don't know what to do about it. I was sniffing all the time in the past few days and it was getting very annoying", "One of my nausal was blocked and I couldn't breath properly", 'My throat hurts after a cold shower last night, I am not sure what to do about it', "It feels like there was something in my throat and I've been dry coughing for several days already", 'I recently developed a condition that requires frequent washroom visits througout the day', 'I am so tired this morning, every single one of my muscle hurts so much', 'All my joints are sour since last morning', 'I feel so cold all the time, I sneezed all the time', 'I am not so conformatble', 'I caught a cold', 'I am shiverring', 'My throat hurts', "I an tired all the time and I don't want to move", 'Runny or stuffy nose', 'sore throat', 'cough', 'congestion', 'body aches', 'sneezing']
WORD_LIST_NEW: ['hi i have a runny nose all the time and i dont know what to do about

In [6]:
# # lets take 80% data as training and remaining 20% for test.
# train_percentage = 0.8

# train_dict = {}
# test_dict = {}
# for i in range(0,len(documents)-1):
#     length = len((documents[i])[0])
#     train_dict[(documents[i])[1]] = ((documents[i])[0])[:length]
#     test_dict[(documents[i])[1]] = ((documents[i])[0])[length:]

# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("TOKENIZED WORDS: ",pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    print("pattern_words:",pattern_words)
    print("BAG: ",bag)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

TOKENIZED WORDS:  ['hi', 'i', 'have', 'a', 'runny', 'nose', 'all', 'the', 'time', 'and', 'i', 'dont', 'know', 'what', 'to', 'do', 'about', 'it', 'i', 'was', 'sniffing', 'all', 'the', 'time', 'in', 'the', 'past', 'few', 'days', 'and', 'it', 'was', 'getting', 'very', 'annoying', 'one', 'of', 'my', 'nausal', 'was', 'blocked', 'and', 'i', 'couldnt', 'breath', 'properly', 'my', 'throat', 'hurts', 'after', 'a', 'cold', 'shower', 'last', 'night', 'i', 'am', 'not', 'sure', 'what', 'to', 'do', 'about', 'it', 'it', 'feels', 'like', 'there', 'was', 'something', 'in', 'my', 'throat', 'and', 'ive', 'been', 'dry', 'coughing', 'for', 'several', 'days', 'already', 'i', 'recently', 'developed', 'a', 'condition', 'that', 'requires', 'frequent', 'washroom', 'visits', 'througout', 'the', 'day', 'i', 'am', 'so', 'tired', 'this', 'morning', 'every', 'single', 'one', 'of', 'my', 'muscle', 'hurts', 'so', 'much', 'all', 'my', 'joints', 'are', 'sour', 'since', 'last', 'morning', 'i', 'feel', 'so', 'cold', 'all'

In [7]:
model = Sequential()
model.add(Dense(512, input_shape=[len(train_x[0])]))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0])))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(np.array(train_x), np.array(train_y),
                    batch_size=16,
                    verbose=1,
                    validation_split=0.1)


# # reset underlying graph data
# tf.reset_default_graph()
# # Build neural network
# net = tflearn.input_data(shape=[None, len(train_x[0])])
# net = tflearn.fully_connected(net, 200)
# net = tflearn.fully_connected(net, 100)
# net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
# net = tflearn.regression(net)

# # Define model and setup tensorboard
# model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# # Start training (apply gradient descent algorithm)
# model.fit(train_x, train_y, n_epoch=10000, batch_size=16, show_metric=True)
# model.save('model.tflearn')

# # evaluate a neural network model
# def evaluate_model(train_x, train_y, test_x, test_y):
#     scores = []
#     n_repeats = 30
#     n_words = test_x.shape[1]
#     for i in range(n_repeats):
#         # define network
# 		model = Sequential()
# 		model.add(Dense(50, input_shape=(n_words,), activation='relu'))
# 		model.add(Dense(1, activation='sigmoid'))
# 		# compile network
# 		model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 		# fit network
# 		model.fit(train_x, train_y, epochs=50, verbose=2)
# 		# evaluate
# 		loss, acc = model.evaluate(test_x, test_y, verbose=0)
# 		scores.append(acc)
# 		print('%d accuracy: %s' % ((i+1), acc))
# 	return scores

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           269312      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           262656      dropout_1[0][0]                  
___________________________________________________________________________________________

In [8]:
# creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

In [9]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y, 'classes_details':classes_details}, open( "training_data", "wb" ) )

In [10]:
import coremltools

coreml_model = coremltools.converters.keras.convert(model)
print("successfully loaded keras model")

coreml_model.save('HeightWeight_model.mlmodel')
#coremltools.utils.save_spec(coreml_model, 'my_model.mlmodel')
print("successfully saved coreML model")

ImportError: No module named 'coremltools'